## Imports and setup

In [ ]:

import requests as req
import urllib3
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt


# disable warnings for unverified HTTPS requests
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set Matplotlib defaults
plt.style.use("ggplot")
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

%config InlineBackend.figure_format = 'retina'

## Data Prep

In [ ]:
# Pick a stock
ticker = input("Enter ticker: ")

# Get the data

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey=6C3GUKE8Z5C6Z2B5"
r = req.get(url, verify=False)
data = r.json()

In [ ]:
# Convert the data to a DataFrame
df = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index').reset_index()
df.rename(columns = {'1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close':'Close','5. volume': 'Volume','index':'Date'}, inplace = True)
df

In [ ]:
# check the data for any missing values
df.isnull().sum()

In [ ]:
# Check data types of all columns
data_types = df.dtypes

# Print data types
print(data_types)

In [ ]:
# Convert date type
df['Date'] = pd.to_datetime(df['Date'])

# convert other
# columns to numeric
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close'] = pd.to_numeric(df['Close'])
df['Volume'] = pd.to_numeric(df['Volume'])

In [ ]:
# Create a visualisation of the data
plt.plot(df['Date'], df['Close'])
plt.show()


In [ ]:
# Add a column for the time
df = df.sort_values(by='Date').reset_index(drop=True)
df2 = df.copy()
df2['Time'] = np.arange(len(df.index))
df2.head()

## Model training

In [ ]:
# Training data
X = df2.loc[:, ['Time']]  # features
y = df2.loc[:, 'Close']  # target

# Train the model
model = LinearRegression()
model.fit(X, y)

# Store the fitted values as a time series with the same time index as
# the training data
y_pred = pd.Series(model.predict(X), index=X.index)

In [ ]:
# Visualize the model preidctions
x = df['Date']
plt.scatter(x, y,color='g')
plt.plot(x, y_pred,color='k')

plt.show()